# SIGNAL-seq ADT mapping with kallisto KITE workflow

1) Map the FASTQ data to barcode reference using the Kallisto KITE pipeline
2) Collapse the polyA and randomHex barcodes into CBs
3) write out the data to a .h5ad file

## Env Setup

### Load packages

In [1]:
# Import packages
import matplotlib
import numpy as np
import pandas as pd
import scanpy as sc
from scipy import sparse, io
import sys, os, argparse

# Import utils
# Get the current working directory
current_dir = os.getcwd()
utils_path = os.path.join(current_dir, '../../', 'utils')
sys.path.append(utils_path)

# Import adt utils functions
from adt_utils import merge_rt_barcodes, pairwise_rt_comparison 

matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

/var/folders/rk/05jx38f57w75573dgxbjlrm40000gn/T/ipykernel_66471/1790902959.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Data input

1) PE FASTQ format input files 
2) ADT antibody panel with barcodes and metadata required to generate mapping index

In [2]:
# Load features reference  
# Clean up the antibody names for problematic characters
df = pd.read_csv('barcode_layouts/Ex0003_kite_panel.csv')
df['Antigen']=df['Antigen'].str.replace(' ','_')
df['Antigen']=df['Antigen'].str.replace('(','')
df['Antigen']=df['Antigen'].str.replace(')','')
df

,Barcode,ADT_identifier,Antigen,Clone,supplier,Ag_category,Ab_ug
0,AAGGCAGACGGTGCA,1,Rat_IgG_Control,G8.8,BioLegend,control,1.00
1,GGCTGCGCACCGCCT,2,CD44_v4,IM7,BioLegend,extra,0.25
2,CGTCCTAGGACATAT,5,pP120-Catenin_[T310],22/p120 (pT310),BD Biosciences,signalling,0.50
3,CTCCAGCTCGAGCTC,6,pRB_[S807/811]_v1,J112-906,BD Biosciences,signalling,1.00
4,AGCGAACCGCCGCTG,8,me2_Histone_H3_[K4],C64G9,CST,signalling,0.27
5,GTGCAAGAGTTGGCG,9,cCaspase_3_[D175]_v2,D3E9,CST,signalling,0.20
6,TGGTGACAAGTATCT,11,Mouse_IgG_Control,MOPC21,BioLegend,control,1.00
7,GCCAAGATCAGGTCC,14,pPDPK1_[S241],J66-653.44.22,BD Biosciences,signalling,0.90
8,TTAGGTGTACACGTT,15,pMKK4/SEK1_[S257],C36C11,CST,signalling,1.40
9,GCATTGCGTCAGGCT,16,pBTK_[Y551]_1_v2,24a/BTK,BD Biosciences,signalling,2.00


In [3]:
# Generate the tsv raw feature file to input into Kallisto - KITE
df[['Barcode', 'Antigen']].to_csv('barcode_layouts/features.tsv', index=None, header=None, sep='\t')
!cat barcode_layouts/features.tsv

AAGGCAGACGGTGCA	Rat_IgG_Control
GGCTGCGCACCGCCT	CD44_v4
CGTCCTAGGACATAT	pP120-Catenin_[T310]
CTCCAGCTCGAGCTC	pRB_[S807/811]_v1
AGCGAACCGCCGCTG	me2_Histone_H3_[K4]
GTGCAAGAGTTGGCG	cCaspase_3_[D175]_v2
TGGTGACAAGTATCT	Mouse_IgG_Control
GCCAAGATCAGGTCC	pPDPK1_[S241]
TTAGGTGTACACGTT	pMKK4/SEK1_[S257]
GCATTGCGTCAGGCT	pBTK_[Y551]_1_v2
CTGGTCGAACTTCGT	p4E-BP1_[T37/46]_v2
AGAGCTAGGATCGGA	pAKT_[T308]
ATAATCATTACGTGG	pNF-κB_p65_[S529]
CGTCGCAATCCATTG	pP38_MAPK_[T180/Y182]_v2
CAAGCGCGGCTTCCG	pHistone_H2A.X_[S139]_3
AGACAGTGATGTCCG	pS6_[S240/S244]_2
CGTACATGTACCACA	Pan-CK
GGACGAGTTCAACGT	CK18
GCGGCTCTGTTGATG	Vimentin
TTGTCACGGTAATAA	Cyclin_B1_2
ATCGAACCGACAGAG	pNDRG1_[T346]
GGTCGACTAGGTCGG	pHistone_H3_[Ser28]_v3
CTCAAGCATTATCAT	Rabbit_IgG_Control


In [4]:
# Use kb to generate the mismatch kallisto index.
!kb ref -i barcode_layouts/mismatch.idx -f1 barcode_layouts/mismatch.fa -g barcode_layouts/t2g.txt --workflow kite barcode_layouts/features.tsv --overwrite

[2024-02-10 18:42:47,019]    INFO [ref_kite] Generating mismatch FASTA at barcode_layouts/mismatch.fa
[2024-02-10 18:42:47,037]    INFO [ref_kite] Creating transcript-to-gene mapping at barcode_layouts/t2g.txt
[2024-02-10 18:42:47,044]    INFO [ref_kite] Indexing barcode_layouts/mismatch.fa to barcode_layouts/mismatch.idx


## Run kallisto and bustools to feature count matrix in H5AD format


In [5]:
# Run kb count pipeline
#%%time
!kb count --h5ad -i barcode_layouts/mismatch.idx -o split_adt/ -w barcode_layouts/split_seqv2_barcode_wlist.txt -g barcode_layouts/t2g.txt -x 1,10,18,1,48,56,1,78,86:1,0,10:0,0,0 --workflow kite -t 2 --keep-tmp --overwrite\
../../../raw_data/fastq/adt/ex0003-adt-76_S1_L001_R1_001.fastq.gz ../../../raw_data/fastq/adt/ex0003-adt-76_S1_L001_R2_001.fastq.gz


[2024-02-10 18:42:57,099]    INFO [count] Using index barcode_layouts/mismatch.idx to generate BUS file to split_adt/ from
[2024-02-10 18:42:57,099]    INFO [count]         ../../../raw_data/fastq/adt/ex0003-adt-76_S1_L001_R1_001.fastq.gz
[2024-02-10 18:42:57,099]    INFO [count]         ../../../raw_data/fastq/adt/ex0003-adt-76_S1_L001_R2_001.fastq.gz
[2024-02-10 18:43:57,623]    INFO [count] Sorting BUS file split_adt/output.bus to split_adt/tmp/output.s.bus
[2024-02-10 18:44:12,458]    INFO [count] Inspecting BUS file split_adt/tmp/output.s.bus
[2024-02-10 18:44:17,852]    INFO [count] Correcting BUS records in split_adt/tmp/output.s.bus to split_adt/tmp/output.s.c.bus with whitelist barcode_layouts/split_seqv2_barcode_wlist.txt
[2024-02-10 18:44:27,030]    INFO [count] Sorting BUS file split_adt/tmp/output.s.c.bus to split_adt/output.unfiltered.bus
[2024-02-10 18:44:35,074]    INFO [count] Generating count matrix split_adt/counts_unfiltered/cells_x_features from BUS file split_adt/

In [6]:
# Use bustools to capture the reads based on umi, ADT_barcode or split_barcode.
# Can filter based on whitelist here too, if needed in the future
!bustools text -o split_adt/bus_text_raw.txt split_adt/output.bus 
!bustools text -o split_adt/bus_text_pp.txt split_adt/output.unfiltered.bus 

Read in 15895179 BUS records
Read in 3212531 BUS records


## Generate anndata object for preprocessing

In [7]:
# Figure output directory
sc.settings.figdir = 'pre_processing_figures'

#  kallisto adt x barocde data
adata = sc.read_h5ad('split_adt/counts_unfiltered/adata.h5ad')
adata

AnnData object with n_obs × n_vars = 93597 × 23
    var: 'feature_name'

In [8]:
# Generate QC counts for CBs
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=True, inplace=True)

## Collapse RT and Random Hex barcodes (BC 1)

### Assign barcodes to well_ids and store as anndata .obs

In [9]:
# Initialize a list for each cell barcode (ubc) component
# RT1
ubc_1 = []
# L2
ubc_2 = []
#L3
ubc_3 = []
# The both ligation BCs
ubc_23 = []

# Loop through each barcode extracting the subsequence
for barcode in adata.obs.index.values:
  ubc_1.append(barcode[-8:])
  ubc_2.append(barcode[8:16])
  ubc_3.append(barcode[:8])
  ubc_23.append(barcode[:16])
  
  

# Add barcode subsequence to adata .obs
adata.obs["barcode_1"] = ubc_1
adata.obs["barcode_2"] = ubc_2
adata.obs["barcode_3"] = ubc_3
adata.obs["barcode_2_3"] = ubc_23

# View adata
adata

AnnData object with n_obs × n_vars = 93597 × 23
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'barcode_1', 'barcode_2', 'barcode_3', 'barcode_2_3'
    var: 'feature_name', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

In [10]:
# Read RT BC mapping dataframe contains sample ID numbers
# 1-48 are PolyA, 49-96 are rHex
mapping_table = pd.read_csv("barcode_layouts/barcodes_v2_id_map.csv")
mapping_table.columns = ["ID", "barcode_1", "sample_id"]

mapping_table

,ID,barcode_1,sample_id
0,1,ACTCGTAA,Control
1,2,AAACGATA,Control
2,3,TTACCTCG,Control
3,4,GCCTGCAA,Control
4,5,TGGTATAC,Control
...,...,...,...
91,92,GACAAAGC,GFs MEKi PI3Ki
92,93,GGGCGATG,GFs MEKi PI3Ki
93,94,ATCTATAA,GFs MEKi PI3Ki
94,95,GCCCATGA,GFs MEKi PI3Ki


In [11]:
# Assign RT barcodes sample_id and well index
indices = []
sample_id = []

# Loop through the RT barcodes
for barcode_1 in adata.obs["barcode_1"]:

  # if barcode_1 in mapping dataframe 
  if barcode_1 in mapping_table["barcode_1"].values:
    index_position = mapping_table["barcode_1"] == barcode_1
    indices.append(mapping_table[index_position]["ID"].values[0])
    sample_id.append(mapping_table[index_position]["sample_id"].values[0])

    # else if barcode_1 is invalid provide warning
  else:
    indices.append(-1)
    sample_id.append("invalid")
    print("WLIST ERROR, INVALID INDICES PRESENT!!")

# Annotate adata with corresponding well and sample_id data
adata.obs["index_1"] = indices
adata.obs["sample_id"] = sample_id

In [12]:
# Read mapping daatframe, for Ligation barcodes 2 and 3
mapping_table_l23 = pd.read_csv("barcode_layouts/barcodes_v1.csv")
mapping_table_l23.columns = ["ID", "barcode", "sample_id"]

mapping_table_l23

,ID,barcode,sample_id
0,1,AACGTGAT,NaN
1,2,AAACATCG,NaN
2,3,ATGCCTAA,NaN
3,4,AGTGGTCA,NaN
4,5,ACCACTGT,NaN
...,...,...,...
91,92,GAACAGGC,NaN
92,93,GACAGTGC,NaN
93,94,GAGTTAGC,NaN
94,95,GATGAATC,NaN


In [13]:
# Assing index for L2 barcode
indices = []

# Loop through the LS barcodes
for barcode_2 in adata.obs["barcode_2"]:

  # Assign if valid
  if barcode_2 in mapping_table_l23["barcode"].values:
    index_position = mapping_table_l23["barcode"] == barcode_2
    indices.append(mapping_table_l23[index_position]["ID"].values[0])

    # else if barcode_2 is invalid provide warning
  else:
    indices.append(-1)
    print("WLIST ERROR, INVALID INDICES PRESENT!!")

# Annotate adata with corresponding indices
adata.obs["index_2"] = indices

In [14]:
# Repeat process for L3 barcodes
indices = []
for barcode_3 in adata.obs["barcode_3"]:

  if barcode_3 in mapping_table_l23["barcode"].values:
    index_position = mapping_table_l23["barcode"] == barcode_3
    indices.append(mapping_table_l23[index_position]["ID"].values[0])
  else:
    indices.append(-1)
    print("WLIST ERROR, INVALID INDICES PRESENT!!")

adata.obs["index_3"] = indices

In [18]:
# Write out indexing well counts to files
# unmerged index data for further analysis
rt_index_counts = adata.obs['index_1'].value_counts()
rt_index_counts.to_csv("./pre_processing_figures/data/rt_index_counts.csv")

lig_bc2 = adata.obs['index_2'].value_counts()
lig_bc2.to_csv("./pre_processing_figures/data/lig_bc2_index_counts.csv")

lig_bc3 = adata.obs['index_3'].value_counts()
lig_bc2.to_csv("./pre_processing_figures/data/lig_bc_3_index_counts.csv")


In [19]:
# Check abundance of sample_ID cell barcodes
adata.obs['sample_id'].value_counts()

sample_id
EGF IGF1          50709
GFs MEKi PI3Ki    23021
Control           19867
Name: count, dtype: int64

### Merge PolyA and rHex barcodes

In [20]:

# Get unique barcodes and their counts. barcodes_and_counts store both the sequence and the count for each unique barcode.
barcodes_and_counts = np.unique(adata.obs["barcode_2_3"].values, return_counts = True)

# Store barcodes sequences with 2 counts in adata
two_counts_barcodes_2_3 = barcodes_and_counts[0][barcodes_and_counts[1] == 2]

# Initialize list to store pairs of barcodes with ID difference different to 48
unmatched_barcodes = []

# Iterate through barcodes with 2 counts
for barcode in two_counts_barcodes_2_3:
  # Select rows from adata that have the corresponding barcode_2_3. Calculate index difference
  selection = adata.obs[adata.obs["barcode_2_3"].values == barcode]
  difference = selection["index_1"][0] - selection["index_1"][1]

  # We want to remove the barcode with the higher index. Check ID difference and input each barcode to the combine_distinct_barcode function in the right order.
  if difference == -48:
    adata = merge_rt_barcodes(adata, selection.index[0], selection.index[1])
  elif difference == 48:
    adata = merge_rt_barcodes(adata, selection.index[1], selection.index[0])


/var/folders/rk/05jx38f57w75573dgxbjlrm40000gn/T/ipykernel_66471/2059597726.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  difference = selection["index_1"][0] - selection["index_1"][1]
/opt/miniconda3/envs/scanpy_v1/lib/python3.10/site-packages/scipy/sparse/_index.py:136: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


In [21]:
# Store barcodes_2_3 with multiple counts
multiple_counts_barcodes_2_3 = barcodes_and_counts[0][barcodes_and_counts[1] > 2]

# Loop through the multimatch barcodes
for barcode in multiple_counts_barcodes_2_3:
  index = adata.obs["barcode_2_3"] == barcode
  
  # Compare this barcode to all other barcodes
  matched = pairwise_rt_comparison(adata.obs[index]["index_1"].values)

  # Cerate Matching list
  matching_barcodes = []
  
  # Loop through the RT index pairs barcodes
  for pair in matched:

    # Add index sequences of paired cell barcodes
    matching_barcodes.append(adata.obs[index].index.values[pair])

  # Combine barcodes
  for matching_barcode in matching_barcodes:
    adata = merge_rt_barcodes(adata, matching_barcode[0], matching_barcode[1])



In [22]:
rt_index_counts = adata.obs['index_1'].value_counts()
rt_index_counts.to_csv("split_adt/rt_index_counts_merged.csv")


In [23]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=True, inplace=True)

In [24]:
# Save the adata
adata.write('ex0003_adt_merged_scanpy.h5ad')